In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
#https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz 

!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [3]:
 # Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

In [4]:
# Next we need to create the list of struct fields
schema = [
StructField("marketplace", StringType(), True),
StructField("customer_id", StringType(), True),
StructField("review_id", StringType(), True),
StructField("product_id", StringType(), True),
StructField("product_parent", StringType(), True),
StructField("product_title", StringType(), True),
StructField("product_category", StringType(), True),
StructField("star_rating", IntegerType(), True),
StructField("helpful_votes", IntegerType(), True),
StructField("total_votes", IntegerType(), True),
StructField("vine", StringType(), True),
StructField("verified_purchase", StringType(), True),
StructField("review_headline", StringType(), True),
StructField("review_body", StringType(), True),
StructField("review_date", DateType(), True),
]
schema

[StructField(marketplace,StringType,true),
 StructField(customer_id,StringType,true),
 StructField(review_id,StringType,true),
 StructField(product_id,StringType,true),
 StructField(product_parent,StringType,true),
 StructField(product_title,StringType,true),
 StructField(product_category,StringType,true),
 StructField(star_rating,IntegerType,true),
 StructField(helpful_votes,IntegerType,true),
 StructField(total_votes,IntegerType,true),
 StructField(vine,StringType,true),
 StructField(verified_purchase,StringType,true),
 StructField(review_headline,StringType,true),
 StructField(review_body,StringType,true),
 StructField(review_date,DateType,true)]

In [5]:
final = StructType(fields=schema)
final

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,StringType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,StringType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,DateType,true)))

In [6]:
from pyspark import SparkFiles
url = "https://paulviet-bucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv"), schema=final, sep="\t", header=True, inferSchema=True)
df.count()

1785997

In [7]:
df.na.drop(subset=["star_rating", "helpful_votes", "total_votes", "vine", ]).show() #(truncate=False)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
df.count()

1785997

In [9]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [10]:
# 1 retrieve all the rows where the total_votes count is equal to or greater than 20
votedformore_df = df.filter(col("total_votes") >= 20)
votedformore_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [11]:
votedformore_df.count()

65379

In [12]:
# 2 retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
over50_df = votedformore_df.filter(col("helpful_votes") / col("total_votes") >= 0.5)
over50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [13]:
# 3 Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_df = over50_df.filter(col("vine") == "Y")
vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48771843|R3KKUSGFZWSUIY|B00YXO5UXG|     555176890|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|Quality Chat Headset|Not every situati...| 2015-08-04|
|         US|   53080186|R10FO5UKKVZBK2|B00XO041RQ|     238654494|PDP AG7 True Wire...|     Video Games|          3|    

In [14]:
print("How many Vine reviews? " + str(vine_df.count()))

How many Vine reviews? 94


In [15]:
# 4 Create a new DataFrame or table that retrieves all the rows where a review was not part of the Vine program (paid), vine == 'N'.
notvine_df = over50_df.filter(col("vine") == "N")
notvine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [16]:
print("How many Non-Vine reviews? " + str(notvine_df.count()))

How many Non-Vine reviews? 40471


In [17]:
# 5a Total number of Reviews
print("Total number of Reviews: " + str(over50_df.count()))

Total number of Reviews: 40565


In [18]:
# 5b the number of 5-star reviews 
print("Total number of 5 star Reviews: " + str(over50_df.filter(col("star_rating")== 5).count()))

Total number of 5 star Reviews: 15711


In [19]:
# 5c percentage of 5-star reviews for the two types of review (paid vs unpaid).
print("Paid 5 star reviews  : " + str(vine_df.filter(col("star_rating")== 5).count()))
print("UnPaid 5 star reviews: " + str(notvine_df.filter(col("star_rating")== 5).count()))
print("Percentage of 5 star reviews Paid vs. Unpaid: " + str(vine_df.filter(col("star_rating")== 5).count() / over50_df.filter(col("star_rating")== 5).count() * 100) + "%")


Paid 5 star reviews  : 48
UnPaid 5 star reviews: 15663
Percentage of 5 star reviews Paid vs. Unpaid: 0.30551842658010314%


In [21]:
# Reviews regardless of rating
print("Paid reviews  : " + str(vine_df.count()))
print("UnPaid reviews: " + str(notvine_df.count()))
print("Percentage of reviews Paid vs. Unpaid: " + str(vine_df.count() / over50_df.count() * 100) + "%")

Paid reviews  : 94
UnPaid reviews: 40471
Percentage of reviews Paid vs. Unpaid: 0.23172685812892888%
